In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import sys

if not '../' in sys.path: sys.path.append('../')

from utils import data_utils
from model_config import config
from ved_detAttn import VarSeq2SeqDetAttnModel

In [ ]:
SRC, TRG, train_data, image_train_data, valid_data, test_data, \
encoder_embeddings_matrix, decoder_embeddings_matrix = data_utils.read_data()
x_train, y_train = train_data.src, train_data.trg
x_val, y_val = valid_data.src, valid_data.trg
x_test, y_test = test_data.src, test_data.trg

# Re-calculate the vocab size based on the word_idx dictionary
config['encoder_vocab'] = len(SRC.vocab)
config['decoder_vocab'] = len(TRG.vocab)

config['image_size'] = 48
    

In [ ]:
model = VarSeq2SeqDetAttnModel(config, 
                               encoder_embeddings_matrix, 
                               decoder_embeddings_matrix, 
                               input_word_index=SRC.voacb, 
                               output_word_index=TRG.vocab)

In [ ]:
if config['load_checkpoint'] != 0: 
    checkpoint = config['model_checkpoint_dir'] + str(config['load_checkpoint']) + '.ckpt'
else:
    checkpoint = tf.train.get_checkpoint_state(os.path.dirname('models/checkpoint')).model_checkpoint_path

img_x_test = data_utils.build_image_mask(batch_size=1000, image_size=config['image_size'])
preds = model.predict(checkpoint, 
                      x_test, 
                      img_x_test,
                      y_test, 
                      y_test, 
                      )

In [ ]:
count = 100
model.show_output_sentences(preds[:count], 
                            y_test[:count], 
                            input_test[:count], 
                            true_test[:count], 
                            )

In [ ]:
model.get_diversity_metrics(checkpoint, x_test, y_test)